<a href="https://colab.research.google.com/github/Surichn/DSAI/blob/main/DS/TA/NLP/NLP5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Word Embeddings for Sentiment Analysis

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

2.8.0


Download imdb_review dataset

In [ ]:
data,info = tds.load("imdb_reviews", with_info= True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQMX5SQ/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQMX5SQ/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


Train and Test Datasets

In [ ]:
train_data, test_data = data['train'], data['test']

# Create empty lists
train_sentences =[]
test_sentences = []

train_labels =[]
test_labels = []

In [ ]:
# Extract Sentences and Labels

for sent, label in train_data:
  train_sentences.append(str(sent.numpy().decode('utf8')))
  train_labels.append(label.numpy())

for sent, label in test_data:
  test_sentences.append(str(sent.numpy().decode('utf8')))
  test_labels.append(label.numpy())

In [ ]:
# Convering label list to arrays
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

Data Preparation

In [ ]:
# Define parameters
vocab_size = 5000
embedding_dim = 16
max_length = 150
trunc_type = 'post'
oov_tok="<oov>"

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

# Training Sequences and labels
train_seqs = tokenizer.texts_to_sequences(train_sentences)
train_pad = pad_sequences(train_seqs, maxlen=max_length,truncating=trunc_type)

# Testing Sequences and lables
test_seqs = tokenizer.texts_to_sequences(test_sentences)
test_pad = pad_sequences(test_seqs, maxlen=max_length)

In [ ]:
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i,'?') for i in text])

print(train_sentences[2])
print(train_pad[2])
print(decode_review(train_pad[2]))

Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Mountie telling the people of Dawson City, Yukon to elect themselves a marshal (yes a marshal!) and to enforce the law themselves, then gunfighters battling it out on the streets for control of the town? <br /><br />Nothing even remotely resembling that happened on the Canadian side of the border during the Klondike gold rush. Mr. Mann and company appear to have mistaken Dawson City for Deadwood, the Canadian North for the American Wild West.<br /><br />Canadian viewers be prepared for a Reefer Madness type of enjoyable howl with this ludicrous plot, or, to shake your head in disgust.
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0 4383    1    2    1 4802 3976    9    4  895 1595
    3 2003 1341    3 2339    1  200  735  352   15   34  208  304    6

Define the Neural network with Embedding layer

*   Use the Sequential API
*   Add an embedding input layer of input size equal to vocabulary size
*   Add a flattern layer and 2 dense layers

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(6,activation = 'relu'),
                             tf.keras.layers.Dense(1,activation = 'sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 150, 16)           80000     
                                                                 
 flatten_3 (Flatten)         (None, 2400)              0         
                                                                 
 dense_3 (Dense)             (None, 6)                 14406     
                                                                 
 dense_4 (Dense)             (None, 1)                 7         
                                                                 
Total params: 94,413
Trainable params: 94,413
Non-trainable params: 0
_________________________________________________________________


Model Training

In [ ]:
num_epochs = 10

model.fit(
    train_pad,
    train_labels,
    epochs = num_epochs,
    validation_data = (test_pad, test_labels)
)

Epoch 1/10
782/782 [==============================] - 5s 5ms/step - loss: 0.4787 - accuracy: 0.7509 - val_loss: 0.3162 - val_accuracy: 0.8629
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2547 - accuracy: 0.8990 - val_loss: 0.3497 - val_accuracy: 0.8465
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1296 - accuracy: 0.9609 - val_loss: 0.4012 - val_accuracy: 0.8410
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0419 - accuracy: 0.9934 - val_loss: 0.4791 - val_accuracy: 0.8405
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0123 - accuracy: 0.9987 - val_loss: 0.5606 - val_accuracy: 0.8373
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.6680 - val_accuracy: 0.8401
Epoch 8/10
782/782 [==============================] - 3s 4ms/step - loss: 7.3902e-04 - accuracy: 1.0000 - val_loss: 0.7123 - val_accuracy: 0.8402
Ep

Deriving weights from the Embedding layer

In [ ]:
l1 = model.layers[0]

weights = l1.get_weights()[0]

print(weights.shape) # Vocabulary size and embedding dimension
 
print(weights)


(5000, 16)
[[-0.02376101 -0.00657261  0.02126918 ... -0.02870765 -0.0162551
  -0.00176714]
 [ 0.01049525 -0.03443856  0.01813498 ...  0.01715639 -0.00099883
  -0.03442954]
 [ 0.03312459 -0.06411345  0.04076331 ... -0.04851519 -0.01249468
   0.0213547 ]
 ...
 [-0.01607364 -0.05707335  0.0809749  ... -0.02901013 -0.1426076
  -0.11932342]
 [-0.06310885 -0.02073101 -0.02003602 ...  0.01475882 -0.13170226
   0.04995724]
 [ 0.0495584   0.15047508  0.12982196 ... -0.00415287  0.00343969
   0.06600015]]


Downloading Vectors and Metadata

In [ ]:
import io

# open text stream for Vectors
vectors = io.open('vectors.tsv','w',encoding='utf-8')

# open text stream for metadata
meta = io.open('meta.tsv', 'w', encoding='utf-8')

# write each word and its corresponding embedding

for index in range(1,vocab_size):
  word = reverse_word_index[index] # flipping key value in the word index
  embeddings = weights[index]

  meta.write(word + "\n")
  vectors.write('\t'.join([str(x) for x in embeddings]) + "\n")

vectors.close()
meta.close()

Download written files to the local machine

In [ ]:
try:
  from google.colab import files

except ImportError:
  pass
else:
  files.download('vectors.tsv')
  files.download('meta.tsv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>